In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.request('GET', 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
all_articles = soup.find_all('article', attrs={'class': 'product_pod'})
len(all_articles)

20

In [4]:
all_articles[0]

<article class="product_pod">
<div class="image_container">
<a href="../../../sharp-objects_997/index.html"><img alt="Sharp Objects" class="thumbnail" src="../../../../media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg"/></a>
</div>
<p class="star-rating Four">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="../../../sharp-objects_997/index.html" title="Sharp Objects">Sharp Objects</a></h3>
<div class="product_price">
<p class="price_color">£47.82</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [5]:
price = all_articles[0].find('p', attrs={'class': 'price_color'})
print(float(price.get_text().split('£')[1]))

book_name = all_articles[0].find('h3').find('a').get('title')
print(book_name)

rating = all_articles[12].find('p').get('class')[1]
print(rating)

47.82
Sharp Objects
Three


In [6]:
def get_all_books_categories_links(link):
    categories = {}
    response = requests.request('GET', link)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_book_links = soup.find('div', attrs={'class': 'side_categories'}).find('ul').find('li').find('ul').find_all('a')
    for book_link in all_book_links:
        true_url = book_link.get('href').replace('../', 'http://books.toscrape.com/catalogue/category/books/')
        if true_url != 'index.html':
            category_name = book_link.get_text().replace(' ', '').replace('\n', '')
            categories[category_name] = true_url
    return categories

In [7]:
import pandas as pd
df = pd.DataFrame(columns=['Book_Name', 'Book_Ratting', 'Book_Price', 'Book_Category'])
df

,Book_Name,Book_Ratting,Book_Price,Book_Category


In [8]:
def get_books(page):
    l = []
    rates = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    response = requests.request('GET', page)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_articles = soup.find_all('article', attrs={'class': 'product_pod'})
    for article in all_articles:
        price = float(article.find('p', attrs={'class': 'price_color'}).get_text().split('£')[1])
        book_name = article.find('h3').find('a').get('title')
        rating = article.find('p').get('class')[1]
        dictt = {'Book_Name' : book_name, 'Book_Ratting' : rating, 'Book_Price' : price}
        l.append(dictt)
    return l

In [9]:
categories = get_all_books_categories_links('http://books.toscrape.com/catalogue/category/books/travel_2/index.html')
for category_name, url in categories.items():
    l = get_books(url)
    for i in range(len(l)):
        dictt = l[i]
        dictt['Book_Category'] = category_name
        df = df.append(dictt, ignore_index=True)

In [10]:
df

,Book_Name,Book_Ratting,Book_Price,Book_Category
0,Sharp Objects,Four,47.82,Mystery
1,"In a Dark, Dark Wood",One,19.63,Mystery
2,The Past Never Ends,Four,56.50,Mystery
3,A Murder in Time,One,16.64,Mystery
4,The Murder of Roger Ackroyd (Hercule Poirot #4),Four,44.10,Mystery
...,...,...,...,...
501,Why the Right Went Wrong: Conservatism--From G...,Four,52.65,Politics
502,Equal Is Unfair: America's Misguided Fight Aga...,One,56.86,Politics
503,Amid the Chaos,One,36.58,Cultural
504,Dark Notes,Five,19.19,Erotica


In [11]:
df.to_csv('Books.csv')

## Good Job!